### AWQ

In [ ]:
from model.modeling_qwen_llama import LlamaForCausalLM
import torch
from model.model_processing import LQ_Tokenizer

model_path = "/home/zhuyao/Sunpeng/llava_qwen/check_point/instruct_525k"
model = LlamaForCausalLM.from_pretrained(model_path,device_map = "cuda:2",torch_dtype=torch.float16,attn_implementation="flash_attention_2",use_cache=False)

min_image_tokens = 4
max_image_tokens = 336
processor_path = "/home/zhuyao/Sunpeng/models/qwen_2B_instruct"
tokenizer_path = "/home/zhuyao/Sunpeng/llava_qwen/tes"

lq_tokenizer = LQ_Tokenizer(tokenizer_path,processor_path,min_image_tokens,max_image_tokens)

quant_path = "/home/zhuyao/Sunpeng/llava_qwen/check_point/AWQ"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}


dataset = [
    [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": "/home/zhuyao/Sunpeng/274e3ef7c2bc430877b5e15d2838e1c1.jpg"},
                {"type": "text", "text": "What does the girl's gentle smile and her white and gold attire convey, and how does the background with its futuristic city and text elements contribute to the overall atmosphere?"},
            ],
        },
        {"role": "assistant", "content": "The girl's gentle smile and white and gold attire suggest an aura of innocence, purity, and warmth, possibly reflecting her role as a kind and virtuous character. The futuristic city in the background, combined with the soft blue tones and vertical text elements, creates a sense of calm and optimism, indicating a harmonious or utopian setting. This combination of character design and setting evokes themes of hope, progress, and serenity."},
    ],
    [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": "/home/zhuyao/Sunpeng/1732782139957.png"},
                {"type": "text", "text": "What is the story behind the girl in the astronaut suit holding flowers in a field of red blooms, with a white cross in front of her and a looming celestial body in the background?"},
            ],
        },
        {"role": "assistant", "content": "The girl in the astronaut suit appears to be visiting a grave marked by a white cross, symbolizing a moment of remembrance or mourning. The vibrant field of red flowers, likely representing poppies, could signify life, memory, or sacrifice. The presence of a looming celestial body in the background adds a sense of cosmic scale and mystery, suggesting that the story may involve themes of loss, exploration, or connection across worlds. The setting hints at a poignant blend of science fiction and emotional depth, possibly linked to a personal or collective tragedy."},
    ]
]


inputs = lq_tokenizer(dataset)

model.quantize(calib_data=inputs, quant_config=quant_config)

model.model.config.use_cache = model.model.generation_config.use_cache = True
model.save_quantized(quant_path, safetensors=True, shard_size="4GB")

The error 'LlamaForCausalLM' object has no attribute 'quantize' is normal because the LLaMA class does not implement this functionality. Since I don't have much knowledge about quantization, I haven't manually implemented it.

However, you can use it through other quantization loading methods.

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = LlamaForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="cuda:2"
)